In [2]:
 # %load_ext rpy2.ipython 
# import vamb
%config IPCompleter.greedy=False
%config Completer.use_jedi=False
# %load_ext rpy2.ipython
%matplotlib inline


In [3]:
import pickle
import numpy as np
import pandas as pd
import os
import glob
import requests
import webbrowser
# import xmltodict
import json
from itertools import compress
import re
import seaborn as sns
import plotly as pl
import matplotlib
import matplotlib.pyplot as plt
from Bio import SeqIO
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import skew
import plotly.io as pio
import plotly.express as px
pio.renderers.default = 'iframe_connected'
plt.rcParams['figure.figsize'] = [10, 10]

In [137]:
from IPython.display import HTML

def View(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))

In [4]:
abd = pd.read_csv("/home/gladkovgpu/soil_institute/trash/GMHI_2020/4347_final_relative_abundances.txt", sep="\t")

In [21]:
abd = abd.set_index("Unnamed: 0")

In [54]:
abd.filter(regex='Healthy').sum(axis=1, numeric_only=True).sort_values().tail(40).to_csv("ind", sep='\t')

In [ ]:
View(pd.DataFrame(res))

In [196]:
import json

files = sorted(glob.glob(f"/home/gladkovgpu/soil_institute/trash/antismash_MH/*/*.json"))
names = [os.path.basename(i).split(".")[0] for i in files]
file_list = list(zip(names, files))

d_mh = {}
for i in file_list:
    with open(i[1]) as file:
        dj = json.load(file)
        d_timing = dj['timings']
        if bool(d_timing):
            df = pd.json_normalize(d_timing)
            d_f = df.to_dict(orient='records')[0]
            d_inner = {}
            for keys, values in d_f.items():
                d_inner[keys.split(".")[-1]] = values
            d_mh.update({i[0]:d_f})
        else:
            pass

files = sorted(glob.glob(f"/home/gladkovgpu/soil_institute/trash/antismash_MN/*/*.json"))
names = [os.path.basename(i).split(".")[0] for i in files]
file_list = list(zip(names, files))

d_mn = {}
for i in file_list:
    with open(i[1]) as file:
        dj = json.load(file)
        d_timing = dj['timings']
        if bool(d_timing):
            df = pd.json_normalize(d_timing)
            d_f = df.to_dict(orient='records')[0]
            d_inner = {}
            for keys, values in d_f.items():
                d_inner[keys.split(".")[-1]] = values
            d_mn.update({i[0]:d_f})
        else:
            pass

files = sorted(glob.glob(f"/home/gladkovgpu/soil_institute/trash/antismash_neutrals/*/*.json"))
names = [os.path.basename(i).split(".")[0] for i in files]
file_list = list(zip(names, files))

d_ne = {}
for i in file_list:
    with open(i[1]) as file:
        dj = json.load(file)
        d_timing = dj['timings']
        if bool(d_timing):
            df = pd.json_normalize(d_timing)
            d_f = df.to_dict(orient='records')[0]
            d_inner = {}
            for keys, values in d_f.items():
                d_inner[keys.split(".")[-1]] = values
            d_ne.update({i[0]:d_f})
        else:
            pass


In [38]:
files = sorted(glob.glob(f"/home/gladkovgpu/soil_institute/trash/antismash_neutrals/*/*.json"))
names = [os.path.basename(i).split(".")[0] for i in files]
file_list = list(zip(names, files))

d_ne = {}

df = None
df_new = None
for i in file_list:
    with open(i[1]) as file:
        dj = json.load(file)
        d_timing = dj['timings']
        if bool(d_timing):
            df_inner = pd.json_normalize(d_timing)
            d_f = df_inner.to_dict(orient='records')[0]
            d_inner = {}
            for keys, values in d_f.items():
                d_inner[keys.split(".")[-1]] = values
            if len(d_f) > 2:
                keylist = [i.split(".")[0] for i in d_f.keys()]
                keylist = set(keylist)
                dict_big = {}
                for a in keylist:
                    l = []
                    colnames = []
                    for key, values in d_f.items():
                        colnames.append(key.split(".")[-1])
                        colnames = list(dict.fromkeys(colnames)) 
                        if key.split(".")[0] == a:
                            l.append(values)
                    dict_big.update({a:l})
                    if df is None:
                        df = pd.DataFrame.from_dict(dict_big, orient='index')
                        df.columns = colnames
                    else:
                        df_new = pd.DataFrame.from_dict(dict_big, orient='index')
                        df_new.columns = colnames
        else:
            pass
        
        if df_new is not None:  
            df_new = df_new.sum(axis=0).to_frame().T
            df_new.index = [i[0]]
            df = pd.concat([df, df_new], axis=0)
        # d_mh.update({i[0]:d_f})
# d_mh
        else:
            df = df.sum(axis=0).to_frame().T
            df.index = [i[0]]
            print(i[0])
NE = df.copy()
NE

Akkermansia_muciniphila


,hmm_detection,nrps_pks_domains,genefunctions,tta,lanthipeptides,thiopeptides,nrps_pks,sactipeptides,lassopeptides,t2pks
Akkermansia_muciniphila,14.421617,3.247155,4.762356,0.150254,0.000070,0.000033,0.928677,0.000128,0.000053,0.000042
Alistipes_putredinis,10.323755,0.735339,1.444899,0.044022,0.000051,0.000027,0.000096,0.000020,0.000022,0.000018
Anaerobutyricum_hallii,15.572239,2.639637,4.156751,0.145796,0.000066,0.000025,3.422996,0.000103,0.000059,0.000045
Bacteroides_eggerthii,16.991625,2.421978,4.034743,0.176341,0.000048,0.000028,0.000134,0.000017,0.000018,0.000018
Bacteroides_massiliensis,31.375954,5.464120,7.660819,0.166503,0.000155,0.000070,2.932985,0.000125,0.000077,0.000070
Bacteroides_stercoris,8.567913,0.703973,1.450744,0.019644,0.000044,0.000025,0.000092,0.000018,0.000020,0.000016
Bacteroides_uniformis,18.132747,4.188355,6.183721,0.193873,0.000052,0.000031,3.533366,0.000114,0.000049,0.000042
Barnesiella_intestinihominis,18.132747,4.188355,6.183721,0.193873,0.000052,0.000031,3.533366,0.000114,0.000049,0.000042
Collinsella_aerofaciens,18.132747,4.188355,6.183721,0.193873,0.000052,0.000031,3.533366,0.000114,0.000049,0.000042
Dorea_longicatena,15.392760,1.591085,3.378153,0.010080,0.000092,0.000045,0.000175,0.000040,0.000040,0.000041


In [39]:
files = sorted(glob.glob(f"/home/gladkovgpu/soil_institute/trash/antismash_MN/*/*.json"))
names = [os.path.basename(i).split(".")[0] for i in files]
file_list = list(zip(names, files))

d_mn = {}

df = None
df_new = None
for i in file_list:
    with open(i[1]) as file:
        dj = json.load(file)
        d_timing = dj['timings']
        if bool(d_timing):
            df_inner = pd.json_normalize(d_timing)
            d_f = df_inner.to_dict(orient='records')[0]
            d_inner = {}
            for keys, values in d_f.items():
                d_inner[keys.split(".")[-1]] = values
            if len(d_f) > 2:
                keylist = [i.split(".")[0] for i in d_f.keys()]
                keylist = set(keylist)
                dict_big = {}
                for a in keylist:
                    l = []
                    colnames = []
                    for key, values in d_f.items():
                        colnames.append(key.split(".")[-1])
                        colnames = list(dict.fromkeys(colnames)) 
                        if key.split(".")[0] == a:
                            l.append(values)
                    dict_big.update({a:l})
                    if df is None:
                        df = pd.DataFrame.from_dict(dict_big, orient='index')
                        df.columns = colnames
                    else:
                        df_new = pd.DataFrame.from_dict(dict_big, orient='index')
                        df_new.columns = colnames
        else:
            pass
        
        if df_new is not None:  
            df_new = df_new.sum(axis=0).to_frame().T
            df_new.index = [i[0]]
            df = pd.concat([df, df_new], axis=0)
        # d_mh.update({i[0]:d_f})
# d_mh
        else:
            df = df.sum(axis=0).to_frame().T
            df.index = [i[0]]
            print(i[0])

            
un_df = df.copy()

files = sorted(glob.glob(f"/home/gladkovgpu/soil_institute/trash/antismash_MH/*/*.json"))
names = [os.path.basename(i).split(".")[0] for i in files]
file_list = list(zip(names, files))

d_mh = {}

df = None
df_new = None
for i in file_list:
    with open(i[1]) as file:
        dj = json.load(file)
        d_timing = dj['timings']
        if bool(d_timing):
            df_inner = pd.json_normalize(d_timing)
            d_f = df_inner.to_dict(orient='records')[0]
            d_inner = {}
            for keys, values in d_f.items():
                d_inner[keys.split(".")[-1]] = values
            if len(d_f) > 2:
                keylist = [i.split(".")[0] for i in d_f.keys()]
                keylist = set(keylist)
                dict_big = {}
                for a in keylist:
                    l = []
                    colnames = []
                    for key, values in d_f.items():
                        colnames.append(key.split(".")[-1])
                        colnames = list(dict.fromkeys(colnames)) 
                        if key.split(".")[0] == a:
                            l.append(values)
                    dict_big.update({a:l})
                    if df is None:
                        df = pd.DataFrame.from_dict(dict_big, orient='index')
                        df.columns = colnames
                    else:
                        df_new = pd.DataFrame.from_dict(dict_big, orient='index')
                        df_new.columns = colnames
        else:
            pass
        
        if df_new is not None:  
            df_new = df_new.sum(axis=0).to_frame().T
            df_new.index = [i[0]]
            df = pd.concat([df, df_new], axis=0)
        # d_mh.update({i[0]:d_f})
# d_mh
        else:
            df = df.sum(axis=0).to_frame().T
            df.index = [i[0]]
            print(i[0])

            
MH = df.copy()

Anaerotruncus_colihominis
Bifidobacterium_dentium_ATCC_27678
Alistipes_communis
Alistipes_senegalensis
Bifidobacterium_adolescentis


In [40]:
un_df

,hmm_detection,nrps_pks_domains,genefunctions,tta,lanthipeptides,thiopeptides,nrps_pks,sactipeptides,lassopeptides,t2pks
Bifidobacterium_dentium_ATCC_27678,13.674305,1.137476,1.957478,0.123187,0.000044,0.000025,0.000134,0.000024,0.000017,0.000021
Blautia_producta_ATCC_27340_=_DSM_2950,19.087965,4.710227,7.366133,0.282922,1.346233,0.000117,12.349757,0.000055,0.000025,0.000020
Clostridiales_bacterium_1_7_47FAA,16.541028,1.854520,3.257797,0.012488,0.000076,0.000040,3.534856,0.000117,0.000074,0.000068
Clostridium_clostridioforme_90B1,12.800293,1.336809,2.445503,0.066289,0.000040,0.000020,0.000104,0.000015,0.000018,0.000015
Clostridium_nexile,25.221467,4.660999,8.484494,0.041307,0.000117,0.000062,0.000422,0.000052,0.000054,0.000049
Eggerthella_lenta,15.389081,1.180842,2.722242,0.132338,0.000053,0.000026,0.000188,0.000024,0.000020,0.000018
Enterocloster_bolteae,18.607056,3.472421,4.347836,0.230603,0.000046,0.000020,4.241171,0.000100,0.000048,0.000044
Enterocloster_citroniae,8.040148,0.837318,1.602202,0.005701,0.000040,0.000023,0.000079,0.000021,0.000018,0.000016
Erysipelotrichaceae_bacterium_2_2_44A,6.674900,0.814612,1.609326,0.012579,0.000039,0.000021,0.000081,0.000016,0.000018,0.000015
Flavonifractor_plautii,14.468354,2.153738,3.521852,0.159257,0.000067,0.000026,0.000150,0.000020,0.000021,0.000019


In [41]:

h_df =  pd.concat([NE, MH], axis = 0)

In [277]:
h_df

,hmm_detection,nrps_pks_domains,genefunctions,tta,lanthipeptides,thiopeptides,nrps_pks,sactipeptides,lassopeptides,t2pks
Akkermansia_muciniphila,14.421617,3.247155,4.762356,0.150254,0.000070,0.000033,0.928677,0.000128,0.000053,0.000042
Alistipes_putredinis,10.323755,0.735339,1.444899,0.044022,0.000051,0.000027,0.000096,0.000020,0.000022,0.000018
Anaerobutyricum_hallii,15.572239,2.639637,4.156751,0.145796,0.000066,0.000025,3.422996,0.000103,0.000059,0.000045
Bacteroides_eggerthii,16.991625,2.421978,4.034743,0.176341,0.000048,0.000028,0.000134,0.000017,0.000018,0.000018
Bacteroides_massiliensis,31.375954,5.464120,7.660819,0.166503,0.000155,0.000070,2.932985,0.000125,0.000077,0.000070
Bacteroides_stercoris,8.567913,0.703973,1.450744,0.019644,0.000044,0.000025,0.000092,0.000018,0.000020,0.000016
Bacteroides_uniformis,18.132747,4.188355,6.183721,0.193873,0.000052,0.000031,3.533366,0.000114,0.000049,0.000042
Barnesiella_intestinihominis,18.132747,4.188355,6.183721,0.193873,0.000052,0.000031,3.533366,0.000114,0.000049,0.000042
Collinsella_aerofaciens,18.132747,4.188355,6.183721,0.193873,0.000052,0.000031,3.533366,0.000114,0.000049,0.000042
Dorea_longicatena,15.392760,1.591085,3.378153,0.010080,0.000092,0.000045,0.000175,0.000040,0.000040,0.000041


In [278]:
h_df.columns

Index(['hmm_detection', 'nrps_pks_domains', 'genefunctions', 'tta',
       'lanthipeptides', 'thiopeptides', 'nrps_pks', 'sactipeptides',
       'lassopeptides', 't2pks'],
      dtype='object')

In [42]:

columnsTitles = ["hmm_detection", "nrps_pks_domains", "genefunctions", "tta", "lanthipeptides", "thiopeptides", "nrps_pks", "sactipeptides", "lassopeptides", "t2pks"]
un_df = un_df.reindex(columns=columnsTitles)

In [298]:
un_df

,hmm_detection,nrps_pks_domains,genefunctions,tta,lanthipeptides,thiopeptides,nrps_pks,sactipeptides,lassopeptides,t2pks
Bifidobacterium_dentium_ATCC_27678,13.674305,1.137476,1.957478,0.123187,0.000044,0.000025,0.000134,0.000024,0.000017,0.000021
Blautia_producta_ATCC_27340_=_DSM_2950,19.087965,4.710227,7.366133,0.282922,1.346233,0.000117,12.349757,0.000055,0.000025,0.000020
Clostridiales_bacterium_1_7_47FAA,16.541028,1.854520,3.257797,0.012488,0.000076,0.000040,3.534856,0.000117,0.000074,0.000068
Clostridium_clostridioforme_90B1,12.800293,1.336809,2.445503,0.066289,0.000040,0.000020,0.000104,0.000015,0.000018,0.000015
Clostridium_nexile,25.221467,4.660999,8.484494,0.041307,0.000117,0.000062,0.000422,0.000052,0.000054,0.000049
Eggerthella_lenta,15.389081,1.180842,2.722242,0.132338,0.000053,0.000026,0.000188,0.000024,0.000020,0.000018
Enterocloster_bolteae,18.607056,3.472421,4.347836,0.230603,0.000046,0.000020,4.241171,0.000100,0.000048,0.000044
Enterocloster_citroniae,8.040148,0.837318,1.602202,0.005701,0.000040,0.000023,0.000079,0.000021,0.000018,0.000016
Erysipelotrichaceae_bacterium_2_2_44A,6.674900,0.814612,1.609326,0.012579,0.000039,0.000021,0.000081,0.000016,0.000018,0.000015
Flavonifractor_plautii,14.468354,2.153738,3.521852,0.159257,0.000067,0.000026,0.000150,0.000020,0.000021,0.000019


In [43]:
scaler = MinMaxScaler()
un_df = pd.DataFrame(scaler.fit_transform(un_df.values), columns=un_df.columns, index=un_df.index)
h_df = pd.DataFrame(scaler.fit_transform(h_df.values), columns=h_df.columns, index=h_df.index)

In [44]:

list_of_rosters = [h_df, un_df]
df_anti = pd.concat(d.assign(type=name)
          for name, d in zip(['healthy_neutral', 'unhealthy'], list_of_rosters))
df_anti = df_anti.drop(columns=["hmm_detection", "genefunctions", "nrps_pks_domains", "nrps_pks"])

In [45]:
df_anti_long = df_anti.melt(id_vars="type",value_name="value", var_name = "cluster_type",ignore_index=False).reset_index()

In [46]:
df_anti_long.to_csv("smash.csv")

In [47]:
df_anti_long

,index,type,cluster_type,value
0,Akkermansia_muciniphila,healthy_neutral,tta,0.708476
1,Alistipes_putredinis,healthy_neutral,tta,0.200755
2,Anaerobutyricum_hallii,healthy_neutral,tta,0.687169
3,Bacteroides_eggerthii,healthy_neutral,tta,0.833155
4,Bacteroides_massiliensis,healthy_neutral,tta,0.786137
...,...,...,...,...
391,Thomasclavelia_ramosa,unhealthy,t2pks,0.086207
392,Veillonella_atypica,unhealthy,t2pks,0.005747
393,[Clostridium]_asparagiforme_DSM_15981,unhealthy,t2pks,0.020115
394,[Clostridium]_symbiosum,unhealthy,t2pks,0.109195


In [48]:
import plotly.express as px

fig = px.bar(df_anti_long, x="index", y="value", color="cluster_type", facet_row="type", title="Antismash results")
# fig.update_layout(barmode='stack', xaxis={'categoryorder':'array', 'categoryarray':['1','2','3','4', '5']})
fig.update_layout(autosize = False,width=1600,height=900)
fig.show()

In [525]:
# files = sorted(glob.glob(f"/home/gladkovgpu/soil_institute/trash/isescan/MN/*/MN/*.sum"))
# names = [os.path.basename(i).split(".")[0] for i in files]
from pathlib import Path


path = f"/home/gladkovgpu/soil_institute/trash/isescan/MN/"
files = Path(path).glob('*/MN/*tsv')

dfs = list()

for f in files:
    data = pd.read_csv(f,sep="\t",skipfooter=1, skiprows=1,engine='python', header = None)
    data['site'] = f.stem.split(".")[0]
    dfs.append(data)

df_mn = pd.concat(dfs, ignore_index=True)
df_mn = df_mn[[1, "site"]]

path = f"/home/gladkovgpu/soil_institute/trash/isescan/neutrals/"
files = Path(path).glob('*/neutrals/*tsv')

dfs = list()

for f in files:
    data = pd.read_csv(f,sep="\t",skipfooter=1, skiprows=1,engine='python', header = None)
    data['site'] = f.stem.split(".")[0]
    dfs.append(data)

df_ne = pd.concat(dfs, ignore_index=True)
df_ne = df_ne[[1, "site"]]

path = f"/home/gladkovgpu/soil_institute/trash/isescan/MH/"
files = Path(path).glob('*/MH/*tsv')

dfs = list()

for f in files:
    data = pd.read_csv(f,sep="\t",skipfooter=1, skiprows=1,engine='python', header = None)
    data['site'] = f.stem.split(".")[0]
    dfs.append(data)

df_mh = pd.concat(dfs, ignore_index=True)
df_mh = df_mh[[1, "site"]]

h_df =  pd.concat([df_mh, df_ne], axis = 0)

list_of_rosters = [h_df, df_mn]
df_is = pd.concat(d.assign(type=name)
          for name, d in zip(['healthy_neutral', 'unhealthy'], list_of_rosters))

df_is

,1,site,type
0,IS1634,Sutterella_wadsworthensis,healthy_neutral
1,IS21,Sutterella_wadsworthensis,healthy_neutral
2,IS1634,Sutterella_wadsworthensis,healthy_neutral
3,IS1634,Sutterella_wadsworthensis,healthy_neutral
4,IS110,Sutterella_wadsworthensis,healthy_neutral
...,...,...,...
1284,IS91,Holdemania_filiformis,unhealthy
1285,IS66,Holdemania_filiformis,unhealthy
1286,IS110,Holdemania_filiformis,unhealthy
1287,IS21,Holdemania_filiformis,unhealthy


In [526]:
df_is["is_type"] = df_is[1]
df_is = df_is.drop(columns=[1])
df_is

,site,type,is_type
0,Sutterella_wadsworthensis,healthy_neutral,IS1634
1,Sutterella_wadsworthensis,healthy_neutral,IS21
2,Sutterella_wadsworthensis,healthy_neutral,IS1634
3,Sutterella_wadsworthensis,healthy_neutral,IS1634
4,Sutterella_wadsworthensis,healthy_neutral,IS110
...,...,...,...
1284,Holdemania_filiformis,unhealthy,IS91
1285,Holdemania_filiformis,unhealthy,IS66
1286,Holdemania_filiformis,unhealthy,IS110
1287,Holdemania_filiformis,unhealthy,IS21


In [527]:
df_is = df_is.groupby(df_is.columns.tolist(),as_index=False).size().sort_values(by='type', ascending=False)
df_is

,site,type,is_type,size
231,Lachnospiraceae_bacterium_1_4_56FAA,unhealthy,IS21,1
251,Lachnospiraceae_bacterium_5_1_57FAA,unhealthy,IS256,1
269,Lachnospiraceae_bacterium_9_1_43BFAA,unhealthy,IS21,3
268,Lachnospiraceae_bacterium_9_1_43BFAA,unhealthy,IS200/IS605,8
267,Lachnospiraceae_bacterium_9_1_43BFAA,unhealthy,IS1634,1
...,...,...,...,...
321,Ruminococcus_bromii,healthy_neutral,IS21,2
322,Ruminococcus_bromii,healthy_neutral,IS256,4
323,Ruminococcus_bromii,healthy_neutral,IS3,2
324,Ruminococcus_bromii,healthy_neutral,IS481,6


In [529]:
import plotly.express as px

fig = px.bar(df_is, x="site", y="size", color="is_type", facet_row="type", title="Isoscan results")
# fig.update_layout(barmode='stack', xaxis={'categoryorder':'array', 'categoryarray':['1','2','3','4', '5']})
fig.update_layout(autosize = False,width=1600,height=900)
fig.show()

In [530]:
df_is.to_csv("is.csv")

In [ ]:
path = f"/home/gladkovgpu/soil_institute/trash/anno_MN/"
files = sorted(glob.glob(f"{path}/*.emapper.annotations"))

In [69]:
names = [os.path.basename(i).split(".")[0] for i in files]
file_list = list(zip(names, files))

In [8]:
names = [os.path.basename(i).split(".")[0] for i in files]
file_list = list(zip(names, files))
header = pd.read_csv(file_list[1][1],sep="\t", skiprows=4 ,nrows=0).columns.to_list()
dfs = [pd.read_csv(f[1], sep="\t", comment="#", header=None, names=header).assign(site=f[0]) for f in file_list]
df_anno_mn = pd.concat(dfs, ignore_index=True)

In [9]:
df_anno_mn = df_anno_mn.replace('-', np.nan)

In [10]:
(df_anno_mn.count() / df_anno_mn.count()[0] * 100).round()

#query            100.0
seed_ortholog     100.0
evalue            100.0
score             100.0
eggNOG_OGs        100.0
max_annot_lvl     100.0
COG_category       92.0
Description        92.0
Preferred_name     38.0
GOs                 7.0
EC                 24.0
KEGG_ko            51.0
KEGG_Pathway       29.0
KEGG_Module        21.0
KEGG_Reaction      19.0
KEGG_rclass        18.0
BRITE              51.0
KEGG_TC            12.0
CAZy                1.0
BiGG_Reaction       2.0
PFAMs              89.0
site              100.0
dtype: float64

In [12]:
for i in df_anno_mn["site"].unique().tolist():
    df_anno_mn.loc[df_anno_mn["site"] == i]["KEGG_ko"].dropna().str.split(",").explode().str[3:].to_csv(f"{i}_keggs", sep="\t")

In [15]:
for i in df_anno_mn["site"].unique().tolist():
    !python /home/gladkovgpu/app/MinPath/MinPath.py -ko {i}_keggs -report result_minpath_site_{i} -details detailed_output_minpath_site_{i} > /dev/null

zsh:1: no matches found: [Clostridium]_asparagiforme_DSM_15981_keggs
zsh:1: no matches found: [Clostridium]_symbiosum_keggs
zsh:1: no matches found: [Ruminococcus]_gnavus_ATCC_29149_keggs


/mnt/storage/soil_institute/graphmb


In [4]:
path = f"/home/gladkovgpu/soil_institute/trash/anno_MH/"
files = sorted(glob.glob(f"{path}/*.emapper.annotations"))
names = [os.path.basename(i).split(".")[0] for i in files]
file_list = list(zip(names, files))
header = pd.read_csv(file_list[1][1],sep="\t", skiprows=4 ,nrows=0).columns.to_list()
dfs = [pd.read_csv(f[1], sep="\t", comment="#", header=None, names=header).assign(site=f[0]) for f in file_list]
df_anno_mh = pd.concat(dfs, ignore_index=True)




In [5]:
path = f"/home/gladkovgpu/soil_institute/trash/anno_neutrals/"
files = sorted(glob.glob(f"{path}/*.emapper.annotations"))
names = [os.path.basename(i).split(".")[0] for i in files]
file_list = list(zip(names, files))
header = pd.read_csv(file_list[1][1],sep="\t", skiprows=4 ,nrows=0).columns.to_list()
dfs = [pd.read_csv(f[1], sep="\t", comment="#", header=None, names=header).assign(site=f[0]) for f in file_list]
df_anno_ne = pd.concat(dfs, ignore_index=True)

In [6]:

df_anno_h = pd.concat([df_anno_ne, df_anno_mh], ignore_index=True)
df_anno_h

,#query,seed_ortholog,evalue,score,eggNOG_OGs,max_annot_lvl,COG_category,Description,Preferred_name,GOs,...,KEGG_Pathway,KEGG_Module,KEGG_Reaction,KEGG_rclass,BRITE,KEGG_TC,CAZy,BiGG_Reaction,PFAMs,site
0,WP_065529414.1,349741.Amuc_0002,0.000000e+00,892.0,"COG0593@1|root,COG0593@2|Bacteria,46S6H@74201|...",203494|Verrucomicrobiae,L,it binds specifically double-stranded DNA at a...,dnaA,-,...,"ko02020,ko04112,map02020,map04112",-,-,-,"ko00000,ko00001,ko03032,ko03036",-,-,-,"Bac_DnaA,Bac_DnaA_C,DnaA_N",Akkermansia_muciniphila
1,WP_012419065.1,349741.Amuc_0003,1.090000e-223,616.0,"COG1131@1|root,COG1131@2|Bacteria,46UAZ@74201|...",203494|Verrucomicrobiae,V,"AAA domain, putative AbiEii toxin, Type IV TA ...",-,-,...,-,M00254,-,-,"ko00000,ko00002,ko02000",3.A.1,-,-,"ABC_tran,DUF4162",Akkermansia_muciniphila
2,WP_012419066.1,349741.Amuc_0004,2.490000e-183,510.0,"COG1277@1|root,COG1277@2|Bacteria,46T0W@74201|...",203494|Verrucomicrobiae,S,ABC-2 family transporter protein,-,-,...,-,M00254,-,-,"ko00000,ko00002,ko02000",3.A.1,-,-,"ABC2_membrane_2,ABC2_membrane_3",Akkermansia_muciniphila
3,WP_102746802.1,349741.Amuc_0005,0.000000e+00,1012.0,"COG3225@1|root,COG3225@2|Bacteria,46SRK@74201|...",203494|Verrucomicrobiae,N,ABC-type uncharacterized transport system,-,-,...,-,-,-,-,-,-,-,-,ABC_transp_aux,Akkermansia_muciniphila
4,WP_102746801.1,349741.Amuc_0006,0.000000e+00,1355.0,"2DN50@1|root,32VJ4@2|Bacteria,46V62@74201|Verr...",203494|Verrucomicrobiae,S,Domain of unknown function (DUF4340),-,-,...,-,-,-,-,-,-,-,-,DUF4340,Akkermansia_muciniphila
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74918,WP_021995204.1,1203554.HMPREF1476_01971,3.290000e-154,433.0,"COG2834@1|root,COG2834@2|Bacteria,1PXDV@1224|P...",28216|Betaproteobacteria,M,Participates in the translocation of lipoprote...,lolA,-,...,-,-,-,-,ko00000,-,-,-,LolA,Sutterella_wadsworthensis
74919,WP_016475052.1,1203554.HMPREF1476_01970,6.760000e-248,682.0,"COG2378@1|root,COG2378@2|Bacteria,1P2ZE@1224|P...",28216|Betaproteobacteria,K,WYL domain,-,-,...,-,-,-,-,-,-,-,-,"HTH_11,WYL",Sutterella_wadsworthensis
74920,WP_016475051.1,1203554.HMPREF1476_01969,1.430000e-272,746.0,"COG2129@1|root,COG2129@2|Bacteria,1RB8N@1224|P...",28216|Betaproteobacteria,S,Calcineurin-like phosphoesterase,-,-,...,-,-,-,-,-,-,-,-,Metallophos,Sutterella_wadsworthensis
74921,WP_016475050.1,1203554.HMPREF1476_01968,8.960000e-223,614.0,"COG0697@1|root,COG0697@2|Bacteria,1R6KH@1224|P...",28216|Betaproteobacteria,EG,EamA-like transporter family,-,-,...,-,-,-,-,-,-,-,-,EamA,Sutterella_wadsworthensis


In [7]:
for i in df_anno_h["site"].unique().tolist():
    print(i)

Akkermansia_muciniphila
Alistipes_putredinis
Anaerobutyricum_hallii
Bacteroides_eggerthii
Bacteroides_massiliensis
Bacteroides_stercoris
Bacteroides_uniformis
Barnesiella_intestinihominis
Collinsella_aerofaciens
Dorea_longicatena
Eubacterium_rectale
Eubacterium_siraeum
Faecalibacterium_prausnitzii
Lachnospira_eligens
Parabacteroides_merdae
Phocaeicola_vulgatus
Roseburia_intestinalis
Ruminococcus_bromii
Ruminococcus_sp_5_1_39BFAA
Ruminococcus_torques
Segatella_copri
Alistipes_communis
Alistipes_senegalensis
Bifidobacterium_adolescentis
Bifidobacterium_angulatum_DSM_20098_=_JCM_7096
Bifidobacterium_catenulatum_DSM_16992_=_JCM_1194_=_LMG_11043
Lachnospiraceae_bacterium_8_1_57FAA
Sutterella_wadsworthensis


In [10]:
for i in df_anno_h["site"].unique().tolist():
    df_anno_h.loc[df_anno_h["site"] == i]["KEGG_ko"].dropna().str.split(",").explode().str[3:].to_csv(f"{i}_keggs", sep="\t")

In [11]:
for i in df_anno_h["site"].unique().tolist():
    !python /home/gladkovgpu/app/MinPath/MinPath.py -ko {i}_keggs -report result_minpath_site_{i} > /dev/null

In [18]:
min_sorted

,site,fam0,fam_found,name,rel
103,Roseburia,512,84,ABC transporters,5.81
3473,Ruminococcus,512,76,ABC transporters,2.16
4988,Sutterella,512,68,ABC transporters,5.61
4607,Eubacterium,512,62,ABC transporters,2.58
1758,Anaerobutyricum,512,62,ABC transporters,4.85
...,...,...,...,...,...
1601,Collinsella,63,1,PPAR signaling pathway,0.09
1633,Collinsella,28,1,Fat digestion and absorption,0.09
1622,Collinsella,86,1,Th17 cell differentiation,0.09
1632,Collinsella,25,1,Type I diabetes mellitus,0.09


In [20]:
min_sorted.loc[min_sorted["name"] == "ABC transporters"]

,site,fam0,fam_found,name,rel
103,Roseburia,512,84,ABC transporters,5.81
3473,Ruminococcus,512,76,ABC transporters,2.16
4988,Sutterella,512,68,ABC transporters,5.61
4607,Eubacterium,512,62,ABC transporters,2.58
1758,Anaerobutyricum,512,62,ABC transporters,4.85
4431,Faecalibacterium,512,55,ABC transporters,4.59
2707,Lachnospiraceae,512,53,ABC transporters,4.72
1407,Ruminococcus,512,51,ABC transporters,1.45
1586,Collinsella,512,44,ABC transporters,4.13
1044,Dorea,512,43,ABC transporters,3.54


In [67]:
from pathlib import Path

path = r'/mnt/storage/soil_institute/graphmb'
files = Path(path).glob('MN/result_minpath_site_*')

dfs = list()
for f in files:
    data = pd.read_csv(f,sep="  ",engine='python', names=["hui0", "hui1", "hui2", "hui22", "fam0", "hui3", "fam_found", "hui5", "name"])
    data['site'] = f.stem
    dfs.append(data)

minpath = pd.concat(dfs, ignore_index=True)
minpath = minpath.drop(list(minpath.filter(regex='hui')), axis=1)
cols = minpath.columns.tolist()
cols = cols[-1:] + cols[:-1]
minpath = minpath[cols]
minpath["site"] = minpath["site"].str.split("_").str[3]
minpath = minpath.sort_values("site")
minpath

import plotly.express as px

minpath["rel"] = minpath["fam_found"].div(minpath.groupby('site')["fam_found"].transform('sum'))
minpath["rel"] = minpath["rel"].transform(lambda x: round(x*100, 2))
min_sorted = minpath.sort_values("fam_found", ascending=False).groupby("site").head(3000)
fp = minpath.groupby("name")["rel"].sum().sort_values().tail(20).to_frame().reset_index()["name"].to_list()
minpath = minpath.loc[minpath['name'].isin(fp)]
min_sorted = minpath.sort_values("fam_found", ascending=False).groupby("site").head(3000)


fig = px.bar(min_sorted, x="site", y="rel", color="name", title="major  MetaCyc pathways in 'EVIL' bacteria")
# fig.update_layout(barmode='stack', xaxis={'categoryorder':'array', 'categoryarray':['1','2','3','4', '5']})
fig.update_layout(autosize=False,width=900,height=800)
fig.show()

In [68]:
from pathlib import Path

path = r'/mnt/storage/soil_institute/graphmb'
files = Path(path).glob('HF/result_minpath_site_*')

dfs = list()
for f in files:
    data = pd.read_csv(f,sep="  ",engine='python', names=["hui0", "hui1", "hui2", "hui22", "fam0", "hui3", "fam_found", "hui5", "name"])
    data['site'] = f.stem
    dfs.append(data)

minpath = pd.concat(dfs, ignore_index=True)
minpath = minpath.drop(list(minpath.filter(regex='hui')), axis=1)
cols = minpath.columns.tolist()
cols = cols[-1:] + cols[:-1]
minpath = minpath[cols]
minpath["site"] = minpath["site"].str.split("_").str[3]
minpath = minpath.sort_values("site")
minpath

minpath["rel"] = minpath["fam_found"].div(minpath.groupby('site')["fam_found"].transform('sum'))
minpath["rel"] = minpath["rel"].transform(lambda x: round(x*100, 2))
min_sorted = minpath.sort_values("fam_found", ascending=False).groupby("site").head(3000)
fp = minpath.groupby("name")["rel"].sum().sort_values().tail(20).to_frame().reset_index()["name"].to_list()
minpath = minpath.loc[minpath['name'].isin(fp)]
min_sorted = minpath.sort_values("fam_found", ascending=False).groupby("site").head(3000)


fig = px.bar(min_sorted, x="site", y="rel", color="name", title="major  MetaCyc pathways in 'GOOD'/'NEUTRAL' bacteria")
# fig.update_layout(barmode='stack', xaxis={'categoryorder':'array', 'categoryarray':['1','2','3','4', '5']})
fig.update_layout(autosize=False,width=900,height=800)
fig.show()